# 2.2 収集した時系列の組み立て

時間解像度の異なる系列データを組み合わせてデータを作る。


## データの用意

データ自体は、[BookRepo](https://github.com/PracticalTimeSeriesAnalysis/BookRepo)に格納されている。
BookRepoリポジトリの[generateAggregateData.py](https://github.com/PracticalTimeSeriesAnalysis/BookRepo/blob/master/Ch02/generateAggregateData.py)を使って作成もできる。

```bash
python generateAggregateData.py
```

これで、`data/year_joined.csv`, `data/donations.csv`, `data/emails.csv`が生成される。
しかし、ここで作成される`data/year_joined.csv`は`user`を示す列が無く、indexで代用するということか？？(issueに書いてあったりするかもだが確認してない)

乱数のシードが固定されていないので、上書きすると値が変わるので注意。

## データの組み立て

In [1]:
import numpy as np
import pandas as pd

まずはデータをpandasで読み込む。

データは、サポートリポジトリに格納されていたものを利用する。上記のスクリプトで作成したものだと、`year_joined.csv`のデータ形式が若干異なる。
解釈はできるのだけど、不安なので。

データは以下の3種類。全て、テーブル形式のデータ。

- `year_joined.csv`: 年ごとの会員資格情報
- `emails.csv`: 週ごとの開封メールの総数
- `donations.csv`: 寄付のタイムスタンプデータ

`user`列がkeyになっている。

In [2]:
url_repo = "https://raw.githubusercontent.com/" \
           "PracticalTimeSeriesAnalysis/BookRepo/master/"

yearJoined = pd.read_csv(url_repo + "Ch02/data/year_joined.csv")
emails = pd.read_csv(url_repo + "Ch02/data/emails.csv")
donation = pd.read_csv(url_repo + "Ch02/data/donations.csv")

display(yearJoined.head())
display(emails.head())
display(donation.head())

,user,userStats,yearJoined
0,0,silver,2014
1,1,silver,2015
2,2,silver,2016
3,3,bronze,2018
4,4,silver,2018


,emailsOpened,user,week
0,3.0,1.0,2015-06-29 00:00:00
1,2.0,1.0,2015-07-13 00:00:00
2,2.0,1.0,2015-07-20 00:00:00
3,3.0,1.0,2015-07-27 00:00:00
4,1.0,1.0,2015-08-03 00:00:00


,amount,timestamp,user
0,25.0,2017-11-12 11:13:44,0.0
1,50.0,2015-08-25 19:01:45,0.0
2,25.0,2015-03-26 12:03:47,0.0
3,50.0,2016-07-06 12:24:55,0.0
4,50.0,2016-05-11 18:13:04,1.0


各データの日付をdatetime型に変換する。

後の処理でdatetime型を仮定した処理がされている箇所がある。（donationについてはは書かれているけど、全て必要）

In [3]:
emails.week = pd.to_datetime(emails.week)
donation.timestamp = pd.to_datetime(donation.timestamp)

データに対する仮説を確認する。思い込みは危険。

まずは会員情報が最新のものだけ、つまり、ユーザに対して状態（Status）が一つだけなのかを確認。

In [4]:
print(yearJoined.shape)
print(yearJoined.groupby("user").count().groupby("userStats").count())

(1000, 3)
           yearJoined
userStats            
1                1000


emailデータは開封した数があったときに記録されるのか？つまり、0件開封したレコードはあるのか？

In [5]:
emails[emails.emailsOpened < 1]

,emailsOpened,user,week


In [6]:
emails[emails.user == 998].head()

,emailsOpened,user,week
25464,1.0,998.0,2017-12-04
25465,3.0,998.0,2017-12-11
25466,3.0,998.0,2017-12-18
25467,3.0,998.0,2018-01-01
25468,3.0,998.0,2018-01-08


0件開封したレコードは無く、週が飛んでいることがわかるので、これらのことから、emailの開封が0の場合は記録されていないということがわかる。

期待される週の数と実際のレコード数の差からもemailの開封が0の場合は記録されていないことがわかる。

In [7]:
print((max(emails[emails.user==998].week) - min(emails[emails.user==998].week)).days / 7)
print(emails[emails.user==998].shape)

25.0
(24, 3)


空白を埋めて綺麗なデータを作る。0件の開封というのも重要な情報。

「何もない」というのはすごく重要なので、見落とさないように注意が必要。

In [8]:
complete_idx = pd.MultiIndex.from_product((set(emails.week), set(emails.user)))

In [9]:
all_email = emails.set_index(["week", "user"]).reindex(complete_idx, fill_value=0).reset_index()
all_email.columns = ["week", "user", "emailsOpened"]
all_email.head()

,week,user,emailsOpened
0,2017-10-23,1.0,3.0
1,2017-10-23,3.0,0.0
2,2017-10-23,5.0,1.0
3,2017-10-23,6.0,2.0
4,2017-10-23,9.0,3.0


これで作ったデータは、全てのユーザで同じ期間分。

しかし、ユーザ毎の入会時期が異なる。ここでは、メールの開封が初週からずっと無い期間、及び、最後まで無い期間を未会員期間と仮定する。（本来は入会/退会時期のデータが欲しい）

In [10]:
(all_email.groupby("user").count() != 173).sum()

week            0
emailsOpened    0
dtype: int64

In [11]:
all_email[all_email.user == 998].sort_values("week").head()

,week,user,emailsOpened
57672,2015-02-09,998.0,0.0
23176,2015-02-16,998.0,0.0
36112,2015-02-23,998.0,0.0
74381,2015-03-02,998.0,0.0
60367,2015-03-09,998.0,0.0


ユーザ毎に、（おそらく）会員前のメールを受け取れない状態のレコードを削除する。

In [12]:
cutoff_dates = emails.groupby("user").week.agg(["min", "max"]).reset_index()
cutoff_dates = cutoff_dates.reset_index()
cutoff_dates 

,index,user,min,max
0,0,1.0,2015-06-29,2018-05-28
1,1,3.0,2018-03-05,2018-04-23
2,2,5.0,2017-06-05,2018-05-28
3,3,6.0,2016-12-05,2018-05-28
4,4,9.0,2016-07-18,2018-05-28
...,...,...,...,...
534,534,991.0,2016-10-24,2016-10-24
535,535,992.0,2015-02-09,2015-07-06
536,536,993.0,2017-09-11,2018-05-28
537,537,995.0,2016-09-05,2018-05-28


In [13]:
for _, row in cutoff_dates.iterrows():
    user = row["user"]
    start_date = row["min"]
    end_date = row["max"]
    all_email.drop(all_email[all_email.user == user][all_email.week < start_date].index, inplace=True)
    all_email.drop(all_email[all_email.user == user][all_email.week > end_date].index, inplace=True)

/tmp/ipykernel_728/702818202.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  all_email.drop(all_email[all_email.user == user][all_email.week < start_date].index, inplace=True)
/tmp/ipykernel_728/702818202.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  all_email.drop(all_email[all_email.user == user][all_email.week > end_date].index, inplace=True)


In [14]:
all_email[all_email.user == 998].sort_values("week").head()

,week,user,emailsOpened
27488,2017-12-04,998.0,1.0
8623,2017-12-11,998.0,3.0
32878,2017-12-18,998.0,3.0
15091,2017-12-25,998.0,0.0
43119,2018-01-01,998.0,3.0


## 時系列の構築

寄付金額と電子メールの開封数との関係を見たいので、寄付金額を上記のデータと結合する。

In [15]:
donation.set_index("timestamp", inplace=True)

In [16]:
donation.head()

,amount,user
timestamp,,
2017-11-12 11:13:44,25.0,0.0
2015-08-25 19:01:45,50.0,0.0
2015-03-26 12:03:47,25.0,0.0
2016-07-06 12:24:55,50.0,0.0
2016-05-11 18:13:04,50.0,1.0


週毎の合計金額に変換する。

メール開封数と結合させるため（ダウンサンプリング）。また、瞬間的なタイミングはここでは重要ではないと考えている。

In [17]:
agg_donations = pd.DataFrame(
    donation.groupby("user").apply(lambda df: df.amount.resample("W-MON").sum().dropna())
).reset_index() # DataFrameにしておく
agg_donations.head()

,user,timestamp,amount
0,0.0,2015-03-30,25.0
1,0.0,2015-04-06,0.0
2,0.0,2015-04-13,0.0
3,0.0,2015-04-20,0.0
4,0.0,2015-04-27,0.0


In [18]:
merged_df = pd.DataFrame() # 初期化が必要
for user, user_email in all_email.groupby("user"):
    user_donations = agg_donations[agg_donations.user == user]
    
    user_donations.set_index("timestamp", inplace=True)
    user_email.set_index("week", inplace=True)
    
    user_email = all_email[all_email.user == user]
    user_email.sort_values("week")
    user_email.set_index("week", inplace=True)
    
    df = pd.merge(user_email, user_donations, how="left", left_index=True, right_index=True)
    df.fillna(0)
    
    df["user"] = df.user_x
    merged_df = merged_df.append(df.reset_index()[["user", "week", "emailsOpened", "amount"]])

In [19]:
merged_df.sort_values(["user", "week"], inplace=True)

In [20]:
merged_df[merged_df.user == 998].head(8)

,user,week,emailsOpened,amount
8,998.0,2017-12-04,1.0,NaN
1,998.0,2017-12-11,3.0,NaN
9,998.0,2017-12-18,3.0,NaN
3,998.0,2017-12-25,0.0,NaN
14,998.0,2018-01-01,3.0,NaN
19,998.0,2018-01-08,3.0,50.0
10,998.0,2018-01-15,2.0,NaN
16,998.0,2018-01-22,3.0,NaN


寄付行動に関与するのは、前の週までの電子メールの開封数と考えられるので、ずらしてみる

In [21]:
df = merged_df[merged_df.user == 998]
df["target"] = df.amount.shift(1)
df = df.fillna(0)
df.head(8)

/tmp/ipykernel_728/584734378.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["target"] = df.amount.shift(1)


,user,week,emailsOpened,amount,target
8,998.0,2017-12-04,1.0,0.0,0.0
1,998.0,2017-12-11,3.0,0.0,0.0
9,998.0,2017-12-18,3.0,0.0,0.0
3,998.0,2017-12-25,0.0,0.0,0.0
14,998.0,2018-01-01,3.0,0.0,0.0
19,998.0,2018-01-08,3.0,50.0,0.0
10,998.0,2018-01-15,2.0,0.0,50.0
16,998.0,2018-01-22,3.0,0.0,0.0
